# Data Preprocessing & Feature Engineering Notebook

In [1]:
#Imports
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBRegressor, XGBClassifier
import xgboost as xgb
#conda install -c conda-forge py-xgboost
import geopandas as gpd
from sklearn.preprocessing import OneHotEncoder
import ast
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR, SVC, LinearSVR
from sklearn.compose import ColumnTransformer
import shap

In [22]:
#set this to the root directory of the project
#path_root_dir="./"
#df = pd.read_csv(path_root_dir+"/ev_census_vehicle_data_all.csv")

df= pd.read_csv("/Users/zaramasood/Downloads/ev_census_vehicle_data_all.csv")

In [23]:
#df.head() #(116572, 33)

Data Preprocessing

In [24]:
df.shape

(116572, 33)

In [25]:
#Filing Blanks
df['STATION_NAME'] = df['STATION_NAME'].fillna('Blink Charging Station')
df.loc[df['LATITUDE'].round(6) == 39.208375, 'CITY'] = 'Liberty'
df.loc[df['LATITUDE'].round(6) == 44.976026, 'CITY'] = 'Minneapolis'
df.loc[df['LATITUDE'].round(6) == 38.252739, 'CITY'] = 'Blasdell'
df.loc[df['LATITUDE'].round(6) == 40.740396, 'CITY'] = 'Fresh'

#Drop if no connector type, not worth keeping
df=df[df['EV_CONNECTOR_TYPES'].notnull()] 

#Drop unneeded features
df = df.drop(columns=['Unnamed: 0', 'STREET_ADDRESS', 'STATE','date', 'POWER_KW',
                     'COUNTY_NAME','COUNTY_STATE', 'Year_Month', 'YEAR'
                     ])

In [26]:
#Filing States with Lat & Long
import reverse_geocoder as rg

state_abbr = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

mask = df['STATE_NAME'].isna() & df['LATITUDE'].notna() & df['LONGITUDE'].notna()

if mask.any():
    coords = list(zip(df.loc[mask, 'LATITUDE'], df.loc[mask, 'LONGITUDE']))
    results = rg.search(coords)  
    imputed_states = []
    for r in results:
        if r['cc'] == 'US':
            abbr = state_abbr.get(r['admin1'], None)
            imputed_states.append(abbr)
        else:
            imputed_states.append(None) 

    df.loc[mask, 'STATE_NAME'] = imputed_states

In [27]:
#Standardizing STATE
state_name_to_abbr = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL',
    'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN',
    'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME',
    'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND',
    'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI',
    'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

valid_abbrs = set(state_name_to_abbr.values())

def standardize_state_name(val):
    if pd.isna(val):
        return None
    val = str(val).strip()
    if val in valid_abbrs:
        return val
    return state_name_to_abbr.get(val, None)

df['STATE'] = df['STATE_NAME'].apply(standardize_state_name)

#Drop if no state present
df=df[df['STATE'].notnull()]

In [28]:
#Completing EV Charging Ports one hot
connectors=['EV_LEVEL1_EVSE_NUM', 'EV_LEVEL2_EVSE_CNUM', 'EV_DC_FAST_NUM']
df[connectors] = df[connectors].fillna(0)

In [29]:
# EV_CONNECTOR_TYPES, one hot for imputing later
def standardize_connector(con_type):
    ct = str(con_type).strip().upper()    
    if ct in ['CHADEMO', 'J1772COMBO', 'J3271']:
        return 'CHAdeMO'
    elif ct in ['TESLA']:
        return 'Combo'
    else:
        return 'J1772'

def clean_and_standardize_connectors(x):
    if pd.isna(x):
        return []
    try:
        x_clean = x.replace('""', '"').strip()
        connectors = ast.literal_eval(x_clean)
        return [standardize_connector(c) for c in connectors]
    except Exception:
        return []

df['clean_connectors'] = df['EV_CONNECTOR_TYPES'].apply(clean_and_standardize_connectors)
categories = ['J1772', 'CHAdeMO', 'Combo']
for cat in categories:
    df[f'CTYPE_{cat}'] = df['clean_connectors'].apply(lambda connectors: 1 if cat in connectors else 0)

df = df.drop(columns=['clean_connectors'])

In [30]:
#Impute demographic & registration features based features based on on State
def clean_to_numeric(series):
    return pd.to_numeric(series.astype(str).str.replace(',', '', regex=True), errors='coerce')
numeric_cols_impute = [
    'POPULATION',
    'MEDIAN_INCOME',
    'ELECTRIC_VEHICLE_REG_COUNT',
    'PLUG_IN_HYBRID_VEHICLE_REG_COUNT',
    'HYBRID_ELECTRIC_REG_COUNT'
]
for col in numeric_cols_impute:
    df[col] = clean_to_numeric(df[col])
    
for col in numeric_cols_impute:
    df[col] = df.groupby('STATE')[col].transform(lambda x: x.fillna(x.median()))    

In [31]:
#One Hot encode 10 most frequent EV networks
#replace NaN with 'Unknown'
df['EV_NETWORK'] = df['EV_NETWORK'].fillna('Unknown')

#10 most frequent networks
top_10_networks = df['EV_NETWORK'].value_counts().head(15).index.tolist()

#replace less frequent values with 'Other'
df['EV_NETWORK_clean'] = df['EV_NETWORK'].where(df['EV_NETWORK'].isin(top_10_networks), 'Other')

#one hot
network_dummies = pd.get_dummies(df['EV_NETWORK_clean'], prefix='EV_NETWORK').astype(int)

df = pd.concat([df, network_dummies], axis=1)
df.drop(columns='EV_NETWORK_clean', inplace=True)

#dummy_cols = [col for col in df.columns if col.startswith('EV_NETWORK_')]
#value_counts_from_dummies = df[dummy_cols].sum().sort_values(ascending=False)
#print(value_counts_from_dummies)

EV_NETWORK_ChargePoint Network    52006
EV_NETWORK_Non-Networked          17839
EV_NETWORK_AMPUP                   8829
EV_NETWORK_Blink Network           7461
EV_NETWORK_Other                   7351
EV_NETWORK_Tesla Destination       6728
EV_NETWORK_Tesla                   3622
EV_NETWORK_EV Connect              2309
EV_NETWORK_SHELL_RECHARGE          2082
EV_NETWORK_eVgo Network            1446
EV_NETWORK_Electrify America       1417
EV_NETWORK_FLO                     1273
EV_NETWORK_LOOP                    1043
EV_NETWORK_VIALYNK                  950
EV_NETWORK_RED_E                    868
EV_NETWORK_SWTCH                    788
dtype: int64


Random Forest Imputer

In [37]:
#Random Forest Imputer Imports
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

In [46]:
#EV Pricing
pricing_patterns = {
    'PRICE_per_kwh_rate': r'\$\d+(\.\d+)?\s*(per\s*kwh|/kwh|kwh)',
    'PRICE_per_minute_rate': r'\$\d+(\.\d+)?\s*(per\s*minute|/minute)',
    'PRICE_per_hour_rate': r'\$\d+(\.\d+)?\s*(per\s*hour|/hour|\bhr\b)',
    'PRICE_per_session_fee': r'\$\d+(\.\d+)?\s*(per\s*session|session fee)',
    'PRICE_monthly_fee': r'monthly service fee',
    'PRICE_activation_fee_present': r'activation fee|connection fee',
    'PRICE_parking_fee_present': r'parking fee|/hr parking|per day parking|garage',
    'PRICE_is_free': r'\bfree\b|donations accepted|no cost',
    'PRICE_other_or_unknown': r'unknown|purchase|paid through|not specified|n/a'
}
df['EV_PRICING'] = df['EV_PRICING'].fillna('').str.lower()

for label, pattern in pricing_patterns.items():
    df[label] = df['EV_PRICING'].str.contains(pattern, regex=True).astype(int)

pricing_cols = [col for col in df.columns if col.startswith('PRICE_') and col != 'PRICE_other_or_unknown']
df['PRICE_None'] = (df[pricing_cols].sum(axis=1) == 0).astype(int)


#Summary
category_columns = ['PRICE_is_free', 'PRICE_per_kwh_rate', 'PRICE_per_minute_rate', 'PRICE_per_hour_rate',
    'PRICE_per_session_fee', 'PRICE_monthly_fee', 'PRICE_activation_fee_present',
    'PRICE_parking_fee_present', 'PRICE_other_or_unknown', 'PRICE_None']
summary = df[category_columns].sum().reset_index()
summary.columns = ['Category', 'Total']
summary = summary.sort_values(by='Total', ascending=False)
#summary


In [49]:
#EV Price Imputing
target_labels = ['PRICE_is_free','PRICE_per_kwh_rate',
    'PRICE_per_minute_rate','PRICE_per_hour_rate','PRICE_per_session_fee',
    'PRICE_monthly_fee','PRICE_activation_fee_present','PRICE_parking_fee_present'
]
features= ['EV_LEVEL1_EVSE_NUM', 'EV_LEVEL2_EVSE_CNUM', 'EV_DC_FAST_NUM', 
          'ELECTRIC_VEHICLE_REG_COUNT', 'PLUG_IN_HYBRID_VEHICLE_REG_COUNT',
       'HYBRID_ELECTRIC_REG_COUNT', 'CTYPE_J1772',
       'CTYPE_CHAdeMO', 'CTYPE_Combo', 
       'EV_NETWORK_Blink Network', 'EV_NETWORK_ChargePoint Network',
       'EV_NETWORK_EV Connect', 'EV_NETWORK_Electrify America',
       'EV_NETWORK_FLO', 'EV_NETWORK_LOOP', 'EV_NETWORK_Non-Networked',
       'EV_NETWORK_Other', 'EV_NETWORK_RED_E', 'EV_NETWORK_SHELL_RECHARGE',
       'EV_NETWORK_SWTCH', 'EV_NETWORK_Tesla', 'EV_NETWORK_Tesla Destination',
       'EV_NETWORK_VIALYNK', 'EV_NETWORK_eVgo Network']
df_train = df[df['PRICE_None'] == 0].copy()
X_train = df_train[features]
y_train = df_train[target_labels]
df_missing = df[df['PRICE_None'] == 1].copy()
X_missing = df_missing[features]
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
model.fit(X_train, y_train)

y_pred = model.predict(X_missing)

predicted_labels = pd.DataFrame(y_pred, columns=target_labels, index=df_missing.index)

df.loc[df_missing.index, target_labels] = predicted_labels

category_columns = ['PRICE_is_free', 'PRICE_per_kwh_rate', 'PRICE_per_minute_rate', 'PRICE_per_hour_rate',
    'PRICE_per_session_fee', 'PRICE_monthly_fee', 'PRICE_activation_fee_present',
    'PRICE_parking_fee_present', 'PRICE_other_or_unknown']#, 'PRICE_None']
summary = df[category_columns].sum().reset_index()
summary.columns = ['Category', 'Total']
summary = summary.sort_values(by='Total', ascending=False)
#summary
df.drop(columns='PRICE_None', inplace=True)

In [63]:
#Facility Types
#One Hot encode 10 most frequent Facility Types \
top_15_facility_types = df['FACILITY_TYPE'].value_counts().head(20).index.tolist()
#initialize all one hot columns to 0
for network in top_15_facility_types:
    df[f'FACILITY_TYPE_{network.replace(" ", "_").replace("-", "_")}'] = (
        df['FACILITY_TYPE'] == network).astype(int)
df['FACILITY_TYPE_clean'] = df['FACILITY_TYPE'].where(
    df['FACILITY_TYPE'].isin(top_15_facility_types), 'Other')
#one hot other
network_dummies = pd.get_dummies(df['FACILITY_TYPE_clean'], prefix='FACILITY_TYPE')
df = pd.concat([df, network_dummies], axis=1)
df['FACILITY_TYPE'] = df['FACILITY_TYPE'].fillna('NaN') #exluces unknow from top 10

#x=['FACILITY_TYPE_clean','FACILITY_TYPE_OTHER', 'FACILITY_TYPE_Other']
#df.drop(columns=(x), inplace=True)
#len(df.columns)

In [64]:
#df.isnull().sum()

STATION_NAME                     0
CITY                             0
LATITUDE                         0
LONGITUDE                        0
EV_CONNECTOR_TYPES               0
                                ..
FACILITY_TYPE_PUBLIC             0
FACILITY_TYPE_RESTAURANT         0
FACILITY_TYPE_SHOPPING_CENTER    0
FACILITY_TYPE_STREET_PARKING     0
FACILITY_TYPE_UTILITY            0
Length: 95, dtype: int64

In [65]:
#FACILITY_TYPE Impute
df_train = df[df['FACILITY_TYPE'] != 'NaN'].copy()
X_train = df_train[features]
y_train = df_train['FACILITY_TYPE']
df_train = df_train[df_train['FACILITY_TYPE'].isin(top_15_facility_types)]
X_train = df_train[features]
y_train = df_train['FACILITY_TYPE']
df_missing = df[df['FACILITY_TYPE'] == 'NaN'].copy()
X_missing = df_missing[features]
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_missing)
df.loc[df_missing.index, 'FACILITY_TYPE'] = y_pred
for facility in top_15_facility_types:
    col_name = f'FACILITY_TYPE_{facility.replace(" ", "_").replace("-", "_")}'
    df[col_name] = (df['FACILITY_TYPE'] == facility).astype(int)


In [76]:
#Impute Station Level info
columns_to_impute = [
    'TOTAL_DURATION',
    'CHARGE_DURATION',
    'ENERGY_KWH','NUM_PORTS', 'TOTAL_SESSIONS']

input_features = [
    'EV_LEVEL1_EVSE_NUM',
    'EV_LEVEL2_EVSE_CNUM',
    'EV_DC_FAST_NUM',
    'POPULATION',
    'MEDIAN_INCOME',
    'ELECTRIC_VEHICLE_REG_COUNT',
    'PLUG_IN_HYBRID_VEHICLE_REG_COUNT',
    'HYBRID_ELECTRIC_REG_COUNT','CTYPE_J1772',
       'CTYPE_CHAdeMO', 'CTYPE_Combo']

for col in columns_to_impute:

    df_train = df[df[col].notnull()]
    df_missing = df[df[col].isnull()]

    X_train = df_train[input_features]
    y_train = df_train[col]
    X_missing = df_missing[input_features]

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    predicted_values = model.predict(X_missing)

    df.loc[df_missing.index, col] = predicted_values


One Hot Encodings 

In [118]:
#categorical encoding 
df["season"] = df["Month"].map({12:'winter',1:'winter',2:'winter',3:'spring',4:'spring',5:'spring',
                                 6:'summer',7:'summer',8:'summer',9:'fall',10:'fall',11:'fall'})


categorical_columns = ['STATE','record_count','season', 
                        'Month',
                      ]

#encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

one_hot_encoded = encoder.fit_transform(df[categorical_columns])
one_hot_df = pd.DataFrame(
    one_hot_encoded,
    columns=encoder.get_feature_names_out(categorical_columns),
    index=df.index
)
df_encoded = pd.concat([df, one_hot_df], axis=1)

#df_encoded=pd.concat([df.drop(columns=categorical_columns), one_hot_df], axis=1)
print (df_encoded.shape) #(116012, 167)
df=df_encoded

(116012, 167)


Feature Engineering

In [121]:
# EV penetration rate - Electric vehicles per person
ev_df=df
ev_df['EV_PENETRATION'] = ev_df['ELECTRIC_VEHICLE_REG_COUNT'] / ev_df['POPULATION']

# EVs Growth Rate - year-over-year by state
ev_df = ev_df.sort_values(['STATE'])
ev_df['EV_GROWTH_RATE'] = ev_df.groupby('STATE')['ELECTRIC_VEHICLE_REG_COUNT'].pct_change()
# Impute missing EV_GROWTH_RATE with the median of the same STATE
ev_df['EV_GROWTH_RATE'] = ev_df.groupby('STATE')['EV_GROWTH_RATE'].transform(
    lambda x: x.fillna(x.median())
)
# EVs per Charging Station
ev_df['EV_PER_STATION'] = ev_df['ELECTRIC_VEHICLE_REG_COUNT'] / ev_df['NUM_PORTS']

# Infrastructure Availability Index - Number of chargers per 1,000 registered EVs
ev_df['EV_INFRASTRUCTURE_INDEX'] = (ev_df['NUM_PORTS'] / ev_df['ELECTRIC_VEHICLE_REG_COUNT']) * 1000

# EV Counts for regression stability
ev_df['LOG_EV_REG'] = np.log1p(ev_df['ELECTRIC_VEHICLE_REG_COUNT'])
ev_df.head()

# Adding weights for combination of adoption and infrastructure
ev_df['ADOPTION_DEMAND_SCORE'] = (
    0.5 * ev_df['EV_PENETRATION'].fillna(0) +
    0.3 * ev_df['EV_GROWTH_RATE'].fillna(0) +
    0.2 * (1 / (1 + ev_df['EV_INFRASTRUCTURE_INDEX'].fillna(0)))  # adding less weight to show low infrastructure
)

df=ev_df
#detect overused stations 
df['sessions_per_port'] = df['TOTAL_SESSIONS'] / df['NUM_PORTS']
#increasing average charge time + session count = stress on infrastructure
df['avg_charge_time'] = df['CHARGE_DURATION'] / df['TOTAL_SESSIONS']


Feature Engineering- Demand Score

In [123]:
df['demand_score'] = df['TOTAL_SESSIONS'] / df['NUM_PORTS']
##High & Low demand
q25 = df["demand_score"].quantile(0.25)
q75 = df["demand_score"].quantile(0.75)
df["high_demand"] = (df["demand_score"] >= q75).astype(int)
df["low_demand"] = (df["demand_score"] <= q25).astype(int) #(67291, 30)


In [126]:
df.describe()

,LATITUDE,LONGITUDE,EV_LEVEL1_EVSE_NUM,EV_LEVEL2_EVSE_CNUM,EV_DC_FAST_NUM,Month,POPULATION,MEDIAN_INCOME,record_count,ELECTRIC_VEHICLE_REG_COUNT,...,EV_GROWTH_RATE,EV_PER_STATION,EV_INFRASTRUCTURE_INDEX,LOG_EV_REG,ADOPTION_DEMAND_SCORE,sessions_per_port,avg_charge_time,demand_score,high_demand,low_demand
count,116012.000000,116012.000000,116012.000000,116012.000000,116012.000000,116012.000000,1.160120e+05,116012.000000,116012.000000,1.160120e+05,...,116012.000000,116012.000000,116012.000000,116012.000000,116012.000000,116012.000000,116012.000000,116012.000000,116012.000000,116012.000000
mean,38.167240,-94.913983,0.042858,2.445583,0.732890,8.283815,1.335182e+06,88845.767860,1.002534,2.471349e+05,...,0.001296,117338.237993,0.152806,11.518225,0.618007,475.506183,-4.633280,475.506183,0.251534,0.250147
std,4.945323,18.813493,0.911997,4.323873,2.714028,2.050002,2.183299e+06,23911.815212,0.051127,3.428613e+05,...,0.053465,172851.162738,0.643358,1.360693,2.701290,554.207074,39.691856,554.207074,0.433897,0.433099
min,19.061250,-162.286348,0.000000,0.000000,0.000000,1.000000,4.630000e+02,25425.000000,1.000000,1.000000e+03,...,-0.641026,70.750000,0.001592,6.908755,0.003933,8.745394,-1602.282091,8.745394,0.000000,0.000000
25%,34.096121,-117.260861,0.000000,1.000000,0.000000,9.000000,1.639830e+05,71662.000000,1.000000,4.740000e+04,...,0.000000,16550.000000,0.011905,10.766399,0.221403,193.155578,0.871263,193.155578,0.000000,0.000000
50%,38.896265,-88.208231,0.000000,2.000000,0.000000,9.000000,6.557090e+05,86150.000000,1.000000,8.490000e+04,...,0.000000,33450.000000,0.029895,11.349241,0.276133,241.567559,2.413552,241.567559,0.000000,0.000000
75%,41.924401,-78.081796,0.000000,2.000000,0.000000,9.000000,1.489634e+06,101763.000000,1.000000,1.680000e+05,...,0.000000,84000.000000,0.060423,12.031725,0.483228,564.044850,2.881185,564.044850,1.000000,1.000000
max,64.852466,-66.983140,90.000000,338.000000,120.000000,12.000000,9.848406e+06,178707.000000,4.000000,1.256600e+06,...,1.785714,628300.000000,14.134276,14.043921,266.748231,3623.000000,57.940123,3623.000000,1.000000,1.000000


In [127]:
#df.to_csv('FinalFeaturesDF.csv', index=False)